In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pathlib import Path
# Coneccion base de datos
from sqlalchemy import create_engine
import cx_Oracle

In [14]:
# Conectar con base de datos
import configparser

config = configparser.ConfigParser()

config.read('../config.ini')

# Obtener los datos de conexión a la base de datos
host = config['oracle']['host']
port = config['oracle']['port']
sid = config['oracle']['service_name']
user = config['oracle']['user']
password = config['oracle']['password']

# Configuración de la base de datos
dsn = cx_Oracle.makedsn(host, port, sid)

engine = create_engine(f'oracle+cx_oracle://{user}:{password}@{dsn}')

In [7]:
import requests
from bs4 import BeautifulSoup
import os

# URL de la página web
url = "https://www.minsalud.gov.co/salud/POS/Paginas/unidad-de-pago-por-capitacion-upc.aspx"

# Obtener el contenido de la página web con verificación SSL deshabilitada
response = requests.get(url, verify=False)
html_content = response.content

# Analizar el contenido HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontrar todos los enlaces
links = soup.find_all('a')

# Directorio para guardar los archivos descargados
download_dir = '../downloads'
os.makedirs(download_dir, exist_ok=True)

# Descargar los archivos ZIP
for link in links:
    href = link.get('href')
    if href and href.endswith('Vigencia%202024.zip'):
        file_url = href if href.startswith('http') else f"https://www.minsalud.gov.co{href}"  # Ajusta si es necesario
        file_name = os.path.join(download_dir, os.path.basename(href))
        print(f"Descargando {file_url} a {file_name}")
        zip_response = requests.get(file_url, verify=False)
        with open(file_name, 'wb') as file:
            file.write(zip_response.content)

print("Descarga completada.")


c:\Users\camil\anaconda3\envs\actualizacion_parametricas_upc_py3.11\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.minsalud.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Descargando https://www.minsalud.gov.co/sites/rid/Lists/BibliotecaDigital/RIDE/VP/RBC/Vigencia%202024.zip a ../downloads\Vigencia%202024.zip


c:\Users\camil\anaconda3\envs\actualizacion_parametricas_upc_py3.11\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.minsalud.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Descarga completada.


In [8]:
# Abrir zip y extraer archivos
import zipfile

def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)


extract_zip('../downloads/Vigencia%202024.zip', '../extracted_files')        

# Actualizacion parametrica CIE10

In [2]:
ruta_cie10 = Path(r"D:\Keralty scripts\Automatizaciones-python\actualizacion_parametricas_upc\extracted_files\Vigencia 2024\2024_TR_CIE10_05_IMG_WEB_26Mar2024.xlsb")
df_cie10 = pd.read_excel(ruta_cie10, skiprows=4, engine='pyxlsb')

In [3]:
df_cie10.head()

,VIGENCIA,CODIGO,DESCRIPCION,SEXO,EDAD_LIM_INF,EDAD_LIM_SUP
0,2024,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",A,0,120
1,2024,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",A,0,120
2,2024,A009,"COLERA, NO ESPECIFICADO",A,0,120
3,2024,A010,FIEBRE TIFOIDEA,A,0,120
4,2024,A011,FIEBRE PARATIFOIDEA A,A,0,120


In [5]:
df_cie10.shape

(12634, 6)

In [8]:
# Revisar si hay unicos
df_cie10.loc[df_cie10.duplicated(subset='CODIGO')]

,VIGENCIA,CODIGO,DESCRIPCION,SEXO,EDAD_LIM_INF,EDAD_LIM_SUP


## Estructura tabla TBL_SUF_CIE10_2024

| Columna                                | Tipo         |
|----------------------------------------|--------------|
| Tabla                                  | VARCHAR2(255)|
| CAPÍTULO                              | VARCHAR2(255)|
| NOMBRE CAPÍTULO               | VARCHAR2(255)|
| CIE10                                  | VARCHAR2(255)|
| DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES | VARCHAR2(255)|
| CODIGO_ CIE-10_ TRES_CARACTERES | VARCHAR2(255)|
| DESCRIPCION_CODIGOS_TRES_CARACTERES | VARCHAR2(255)|
| SEXO                                   | VARCHAR2(255)|
| LIMITE_INFERIOR                 | VARCHAR2(255)|
| LIMITE_SUPERIOR                | VARCHAR2(255)|
| GRUPO_ MORTALIDAD_LISTA_ 6-607 | VARCHAR2(255)|
| GRUPO_MORTALIDAD_LISTA_105 | VARCHAR2(255)|
| VALOR_LIM_INF                      | VARCHAR2(255)|
| VALOR_LIM_SUP                     | VARCHAR2(255)|
| TIPO_LIMI_INFERIOR              | VARCHAR2(20)|
| TIPO_LIMI_SUPERIOR             | VARCHAR2(20)|

In [10]:

df_cie10.rename(columns={'VIGENCIA': 'Tabla', 'CODIGO':'CIE10', 'DESCRIPCION': 'DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES', 'EDAD_LIM_INF': 'VALOR_LIM_INF','EDAD_LIM_SUP': 'VALOR_LIM_SUP'}, inplace=True)

In [12]:
df_cie10.columns.to_list()

['Tabla',
 'CIE10',
 'DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES',
 'SEXO',
 'VALOR_LIM_INF',
 'VALOR_LIM_SUP']

In [15]:
# Copiar estructura de la tabla EPSS05LIQUIDACION_202405 para crear la tabla EPSS05LIQUIDACION_202406

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            TRUNCATE TABLE TBL_SUF_CIE10_2024
        """)    
    except Exception as e:
        print(e)
    

# Hacer la insercion por chunk df_liquidados
df_cie10.to_sql('tbl_suf_cie10_2024', con=engine, if_exists='append', index=False)

12634

# Actualizacion REPS

In [35]:
ruta_reps = Path(r"D:\Keralty scripts\Automatizaciones-python\actualizacion_parametricas_upc\extracted_files\Vigencia 2024\2024_REPS_01-2024.xlsb")
df_reps = pd.read_excel(ruta_reps, skiprows=4, engine='pyxlsb')

In [36]:
df_reps

,AÑO_VIGENCIA,CÓDIGO HABILITACION,NOMBRE PRESTADOR,NIT
0,2024,110013833401,KAREN LORENA FARFAN BARAJAS,1000062006
1,2024,110013833402,KAREN LORENA FARFAN BARAJAS,1000062006
2,2024,110013849601,MARIA ALEJANDRA LEON SARMIENTO,1000063184
3,2024,660010022201,Francisco Javier Estrada Chavez,10000753
4,2024,660010092901,LUIS ANGEL SUAREZ MEJIA,10000837
...,...,...,...,...
76353,2024,050011864004,MAURICIO LONDO¥O LONDO¥O,9976194
76354,2024,050011864005,MAURICIO LONDO¥O LONDO¥O,9976194
76355,2024,520010215701,FREDDY ISRAEL PANTOJA CHAMORRO,9976212
76356,2024,170010285301,SANDRO PARRA SOTO,9976701


In [37]:
df_reps.shape

(76358, 4)

In [38]:
df_reps.loc[df_reps.duplicated(subset='CÓDIGO HABILITACION')]

,AÑO_VIGENCIA,CÓDIGO HABILITACION,NOMBRE PRESTADOR,NIT


## Tabla tbl_suf_reps_2024

| Columna        | Tipo         |
|----------------|--------------|
| COD_PRESTADOR  | VARCHAR2(255)|
| NOM_PRESTADOR  | VARCHAR2(255)|
| NIT            | VARCHAR2(255)|

In [39]:
df_reps.rename(columns={'CÓDIGO HABILITACION':'COD_PRESTADOR','NOMBRE PRESTADOR':'NOM_PRESTADOR','NIT':'NIT'},inplace=True)
df_reps.columns

Index(['AÑO_VIGENCIA', 'COD_PRESTADOR', 'NOM_PRESTADOR', 'NIT'], dtype='object')

In [42]:
df_reps.drop(columns=['AÑO_VIGENCIA'], inplace=True)

In [43]:

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            TRUNCATE TABLE tbl_suf_reps_2024
        """)    
    except Exception as e:
        print(e)
    

# Hacer la insercion por chunk df_liquidados
df_reps.to_sql('tbl_suf_reps_2024', con=engine, if_exists='append', index=False)

76358

# Actualizacion CUPS

In [47]:
ruta_cups = Path(r"D:\Keralty scripts\Automatizaciones-python\actualizacion_parametricas_upc\extracted_files\Vigencia 2024\2024_TR_CUPS_06_COBERTURA_Res2366_WM_14May2024.xlsb")
df_cups = pd.read_excel(ruta_cups, skiprows=4, engine='pyxlsb')

In [48]:
df_cups.head()

,AÑO_VIGENCIA,CODIGO,DESCRIPCION,SEXO,AMBITO,ESTANCIA,COBERTURA,DUPLICADO,VIDA,DX_RELACIONADO,
0,2024,010101,"PUNCION CISTERNAL, VIA LATERAL",Z,Z,Z,PBS,D,Z,NaN,NaN
1,2024,010102,"PUNCION CISTERNAL, VIA MEDIAL",Z,Z,Z,PBS,D,Z,NaN,NaN
2,2024,010103,PUNCION CISTERNAL,Z,Z,Z,PBS,Z,Z,NaN,NaN
3,2024,010201,PUNCION (ASPIRACION DE LIQUIDO) VENTRICULAR A ...,Z,Z,Z,PBS,D,Z,NaN,NaN
4,2024,010202,PUNCION (ASPIRACION DE LIQUIDO) VENTRICULAR PO...,Z,Z,Z,PBS,D,Z,NaN,NaN


In [49]:
df_cups.shape

(9991, 11)

In [50]:
df_cups.loc[df_cups.duplicated(subset='CODIGO')]

,AÑO_VIGENCIA,CODIGO,DESCRIPCION,SEXO,AMBITO,ESTANCIA,COBERTURA,DUPLICADO,VIDA,DX_RELACIONADO,


## Tabla tbl_suf_cups_2024

| Columna                        | Tipo         |
|--------------------------------|--------------|
| CODIGO                         | VARCHAR2(255)|
| DESCRIPCION                    | VARCHAR2(500)|
| FINANCIACION 2481 y 163 de 2021| VARCHAR2(255)|
| FINANCIACION Res 2292 de 2021  | VARCHAR2(255)|
| COBERTURA                      | VARCHAR2(255)|
| SEXO                           | VARCHAR2(255)|
| AMBITO                         | VARCHAR2(255)|
| ESTANCIA                       | VARCHAR2(255)|
| DUPLICADO                      | VARCHAR2(255)|
| VIDA                           | VARCHAR2(255)|
| CIE_10 RELACIONADOS            | VARCHAR2(255)|
| PBS                            | VARCHAR2(255)|
| CUPS_HOMOLOGO_PRIMERA_VEZ      | VARCHAR2(255)|
| DESC_HOMOLOGO                  | VARCHAR2(255)|
| INCLUSION                      | VARCHAR2(255)|

In [58]:
df_cups.head()

,CODIGO,DESCRIPCION,SEXO,AMBITO,ESTANCIA,COBERTURA,DUPLICADO,VIDA,DX_RELACIONADO
0,010101,"PUNCION CISTERNAL, VIA LATERAL",Z,Z,Z,PBS,D,Z,NaN
1,010102,"PUNCION CISTERNAL, VIA MEDIAL",Z,Z,Z,PBS,D,Z,NaN
2,010103,PUNCION CISTERNAL,Z,Z,Z,PBS,Z,Z,NaN
3,010201,PUNCION (ASPIRACION DE LIQUIDO) VENTRICULAR A ...,Z,Z,Z,PBS,D,Z,NaN
4,010202,PUNCION (ASPIRACION DE LIQUIDO) VENTRICULAR PO...,Z,Z,Z,PBS,D,Z,NaN


In [57]:
df_cups.drop(columns=['AÑO_VIGENCIA',' '], inplace=True)

In [60]:
df_cups.rename(columns={'DX_RELACIONADO':'CIE_10 RELACIONADOS'}, inplace=True)

In [61]:

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            TRUNCATE TABLE tbl_suf_cups_2024
        """)    
    except Exception as e:
        print(e)
    

# Hacer la insercion por chunk df_liquidados
df_cups.to_sql('tbl_suf_cups_2024', con=engine, if_exists='append', index=False)

9991

## Actualizacion insumos

In [65]:
ruta_insumos = Path(r"D:\Keralty scripts\Automatizaciones-python\actualizacion_parametricas_upc\extracted_files\Vigencia 2024\2024_INSUMOS Y DISPOSITIVOS_01-24.xlsb")
df_insumos = pd.read_excel(ruta_insumos, skiprows=4,engine='pyxlsb')

In [71]:
df_insumos

,CÓDIGO,DESCRIPCIÓN
0,170100,Lentes
1,170200,Monturas
2,D00001,Stent duodenal
3,D00002,Stent coronario recubierto o medicado
4,D00003,Stent coronario convencional o no recubierto
5,D00004,Glucometro entregado a pacientes diabeticos qu...
6,D00005,Tirillas entregado a pacientes diabeticos que ...
7,D00006,Lancetas entregado a pacientes diabeticos que ...
8,D00007,Kit de ostomia (completo)
9,D00041,Glucometro entregado a pacientes diabeticos qu...


In [68]:
df_insumos.shape

(19, 4)

In [69]:
df_insumos.loc[df_insumos.duplicated(subset='CÓDIGO')]

,AÑO_VIGENCIA,CÓDIGO,DESCRIPCIÓN,


In [70]:
df_insumos.drop(columns=['AÑO_VIGENCIA', ' '], inplace=True)

In [73]:
df_insumos.rename(columns={'CÓDIGO':'CODIGO','DESCRIPCIÓN':'DESCRIPCION'}, inplace=True)

In [74]:

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            TRUNCATE TABLE tbl_suf_insumos_2024
        """)    
    except Exception as e:
        print(e)
    

# Hacer la insercion por chunk df_liquidados
df_insumos.to_sql('tbl_suf_insumos_2024', con=engine, if_exists='append', index=False)

19